# Question B 

Download two attached datasets (Fake.csv and True.csv), which contains real and fake news. In the given
dataset, there is a total of five columns. Now build a Bi-LSTM model to detect fake mews using TensorFlow
and other available libraries. Some hints are as follows:

## Import Data

In [8]:
import pandas as pd

true = pd.read_csv("True.csv")
fake = pd.read_csv("Fake.csv")
true.head() #there is a lot of unneeded info - such as subject & date

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [9]:
#keep only the title and text columns 
true_1 = true[["title", "text"]]
fake_1 = fake[["title", "text"]]
fake_1.head() 

,title,text
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk..."
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ..."
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...


In [13]:
# Assign labels based on the truth/fakeness of the text
true_1['class'] = 1 #assign real news a label of 1
fake_1["class"] = 0 #assign fake news a label of 0 

#Weirdly getting an errorbut the code is running. I did this the exact same way...

<ipython-input-13-e73f511925e8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  true_1['class'] = 1 #assign real news a label of 1
<ipython-input-13-e73f511925e8>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fake_1["class"] = 0 #assign fake news a label of 0


In [17]:
# Concatenate both dataframes into 1 
frames= [true_1, fake_1]
df = pd.concat(frames)

# make the title & text a single column 
df["article"] = df["title"] + " "+ df["text"]
df.drop(["title", "text"], axis=1)

,class,article
0,1,"As U.S. budget fight looms, Republicans flip t..."
1,1,U.S. military to accept transgender recruits o...
2,1,Senior U.S. Republican senator: 'Let Mr. Muell...
3,1,FBI Russia probe helped by Australian diplomat...
4,1,Trump wants Postal Service to charge 'much mor...
...,...,...
23476,0,McPain: John McCain Furious That Iran Treated ...
23477,0,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...
23478,0,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...
23479,0,How to Blow $700 Million: Al Jazeera America F...


## Remove Stopwords in Dataset

Stopwords are unnecessary text like "the", "in", "to" and "of" that take away from the gist of a text. 

In [21]:
import nltk.corpus
from nltk.corpus import stopwords

stop_words = stopwords.words("english")
df["article_wo_stopwords"] = df["article"].apply(lambda x: " ".join([word for word in x.split() if word not in (stop_words)]))
df[["article", "article_wo_stopwords"]].head(5)

,article,article_wo_stopwords
0,"As U.S. budget fight looms, Republicans flip t...","As U.S. budget fight looms, Republicans flip f..."
1,U.S. military to accept transgender recruits o...,U.S. military accept transgender recruits Mond...
2,Senior U.S. Republican senator: 'Let Mr. Muell...,Senior U.S. Republican senator: 'Let Mr. Muell...
3,FBI Russia probe helped by Australian diplomat...,FBI Russia probe helped Australian diplomat ti...
4,Trump wants Postal Service to charge 'much mor...,Trump wants Postal Service charge 'much more' ...


## Remove Words with 2 or fewer Characters 

For a better model, remove stopwords and remove words with 2 or fewer characters, after that split data into test and train set. 

In [24]:
#this isn't really working
df["article_wo_smallword"] = df["article_wo_stopwords"].apply(lambda x: " ".join([word for word in x.split() if len(word) > 2]))
df[["article", "article_wo_stopwords", "article_wo_smallword"]].head(-5)
df

,title,text,class,article,Articles,article_wo_stopwords,article_wo_smallword
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,1,"As U.S. budget fight looms, Republicans flip t...","As U.S. budget fight looms, Republicans flip f...","As U.S. budget fight looms, Republicans flip f...","U.S. budget fight looms, Republicans flip fisc..."
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,1,U.S. military to accept transgender recruits o...,U.S. military accept transgender recruits Mond...,U.S. military accept transgender recruits Mond...,U.S. military accept transgender recruits Mond...
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,1,Senior U.S. Republican senator: 'Let Mr. Muell...,Senior U.S. Republican senator: 'Let Mr. Muell...,Senior U.S. Republican senator: 'Let Mr. Muell...,Senior U.S. Republican senator: 'Let Mr. Muell...
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,1,FBI Russia probe helped by Australian diplomat...,FBI Russia probe helped Australian diplomat ti...,FBI Russia probe helped Australian diplomat ti...,FBI Russia probe helped Australian diplomat ti...
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,1,Trump wants Postal Service to charge 'much mor...,Trump wants Postal Service charge 'much more' ...,Trump wants Postal Service charge 'much more' ...,Trump wants Postal Service charge 'much more' ...
...,...,...,...,...,...,...,...
23476,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,0,McPain: John McCain Furious That Iran Treated ...,McPain: John McCain Furious That Iran Treated ...,McPain: John McCain Furious That Iran Treated ...,McPain: John McCain Furious That Iran Treated ...
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,0,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,0,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Sunnistan: US Allied ‘Safe Zone’ Plan Take Ter...,Sunnistan: US Allied ‘Safe Zone’ Plan Take Ter...,Sunnistan: Allied ‘Safe Zone’ Plan Take Territ...
23479,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,0,How to Blow $700 Million: Al Jazeera America F...,How Blow $700 Million: Al Jazeera America Fina...,How Blow $700 Million: Al Jazeera America Fina...,How Blow $700 Million: Jazeera America Finally...


In [28]:
import re
def preprocessing(text):
    text = re.sub("<[^>]*>", "", text)
    text = re.sub(r"[^\w\s]", "", text)
    text=text.lower()
    return text 

df["clean_articles"] = df["article_wo_smallword"].apply(preprocessing)
df["clean_articles"] #check if changes have been applied 

0        us budget fight looms republicans flip fiscal ...
1        us military accept transgender recruits monday...
2        senior us republican senator let mr mueller jo...
3        fbi russia probe helped australian diplomat ti...
4        trump wants postal service charge much more am...
                               ...                        
23476    mcpain john mccain furious that iran treated s...
23477    justice yahoo settles email privacy classactio...
23478    sunnistan allied safe zone plan take territori...
23479    how blow 700 million jazeera america finally c...
23480    us navy sailors held iranian military signs ne...
Name: clean_articles, Length: 44898, dtype: object

### Split into Test and Train Set 

In [29]:
from sklearn.model_selection import train_test_split

X = df["clean_articles"]
y = df["class"]

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 1)

## Tokenization

You might also need to create a tokenizer to tokenize the words and create sequences of tokenized words.

#### Parameters

In [30]:
most_common_size = 10000 #take most common 10,000 words 
max_len = 256 #256 characters per sequence 
unrecognized_words = "OOV" #where OOV stands for "out of vocabulary", used for words that aren't in word index 
padding = "post" #pad article length to 256 (adding 56 zeros)
truncation = "post"

In [33]:
#Set the tokenizer 
import tensorflow as tf
from tensorflow import keras 
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words = most_common_size, oov_token = unrecognized_words) #specify 10,000 most frequent words 
tokenizer.fit_on_texts(X_train) #method that goes through text, creating dictionary 

In [34]:
X_train = tokenizer.texts_to_sequences(X_train) #converts tokens of text corpus to sequence of integers (which is the index of a token in a dictionary)
X_test = tokenizer.texts_to_sequences(X_test)

In [35]:
X_train= tf.keras.preprocessing.sequence.pad_sequences(X_train, padding = padding, truncating = truncation, maxlen = max_len)
X_test= tf.keras.preprocessing.sequence.pad_sequences(X_test, padding = padding, truncating = truncation, maxlen = max_len)

## Building Out the Model 

You can call Bi-Directional LSTM from Keras and where fitting set Sigmoid and ReLU as activation function, adam as the optimizer, and binary cross-entropy loss function should be used. 

In [38]:
model_lstm = tf.keras.Sequential([
    tf.keras.layers.Embedding(most_common_size, 64), 
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences = True)), 
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(16)), 
    tf.keras.layers.Dense(64, activation="relu"), 
    tf.keras.layers.Dropout(0.5), 
    tf.keras.layers.Dense(1)
])

# We probably need to comment on what each layer in the lstm model is doing here. 

model_lstm.summary()

# it seems like sigmoid is missing here. 
# "tf.keras.activations.sigmoid(x)"

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, None, 64)          640000    
                                                                 
 bidirectional_4 (Bidirectio  (None, None, 128)        66048     
 nal)                                                            
                                                                 
 bidirectional_5 (Bidirectio  (None, 32)               18560     
 nal)                                                            
                                                                 
 dense_4 (Dense)             (None, 64)                2112      
                                                                 
 dropout_2 (Dropout)         (None, 64)                0         
                                                                 
 dense_5 (Dense)             (None, 1)                

In [ ]:
#example from medium

def build_bilstm(word_index, embeddings_dict, nclasses,  MAX_SEQUENCE_LENGTH=500, EMBEDDING_DIM=50, dropout=0.5, hidden_layer = 3, lstm_node = 32):
    # Initialize a sequebtial model
    model = Sequential()
    # Make the embedding matrix using the embedding_dict
    embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
    for word, i in word_index.items():
        embedding_vector = embeddings_dict.get(word)
        if embedding_vector is not None:
            # words not found in embedding index will be all-zeros.
            if len(embedding_matrix[i]) != len(embedding_vector):
                print("could not broadcast input array from shape", str(len(embedding_matrix[i])),
                      "into shape", str(len(embedding_vector)), " Please make sure your"
                                                                " EMBEDDING_DIM is equal to embedding_vector file ,GloVe,")
                exit(1)
            embedding_matrix[i] = embedding_vector
            
    # Add embedding layer
    model.add(Embedding(len(word_index) + 1,
                                EMBEDDING_DIM,
                                weights=[embedding_matrix],
                                input_length=MAX_SEQUENCE_LENGTH,
                                trainable=True))
    # Add hidden layers 
    for i in range(0,hidden_layer):
        # Add a bidirectional lstm layer
        model.add(Bidirectional(LSTM(lstm_node, return_sequences=True, recurrent_dropout=0.2)))
        # Add a dropout layer after each lstm layer
        model.add(Dropout(dropout))
    model.add(Bidirectional(LSTM(lstm_node, recurrent_dropout=0.2)))
    model.add(Dropout(dropout))
    # Add the fully connected layer with 256 nurons and relu activation
    model.add(Dense(256, activation='relu'))
    # Add the output layer with softmax activation since we have 2 classes
    model.add(Dense(nclasses, activation='softmax'))
    # Compile the model using sparse_categorical_crossentropy
    model.compile(loss='sparse_categorical_crossentropy',
                      optimizer='adam',
                      metrics=['accuracy'])
    return model

In [39]:
model_lstm.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), 
             optimizer=tf.keras.optimizers.Adam(1e-2), 
             metrics=["accuracy"])

## Batch Size & Epochs

During training set the batch size to 64 and the number of epochs to 2

In [40]:
early_stop =tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience = 2, restore_best_weights=True)

model_lstm.fit(X_train, y_train, epochs = 2, validation_split= 0.1, batch_size = 64, callbacks=[early_stop])

Epoch 1/2
506/506 [==============================] - 155s 297ms/step - loss: 0.0580 - accuracy: 0.9772 - val_loss: 0.0091 - val_accuracy: 0.9983
Epoch 2/2
506/506 [==============================] - 154s 304ms/step - loss: 0.0056 - accuracy: 0.9987 - val_loss: 0.0039 - val_accuracy: 0.9992


In [41]:
model_lstm

In [63]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44898 entries, 0 to 23480
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   title      44898 non-null  object
 1   text       44898 non-null  object
 2   class      44898 non-null  int64 
 3   article    44898 non-null  object
 4   Articles   44898 non-null  object
 5   Articles2  44898 non-null  object
dtypes: int64(1), object(5)
memory usage: 2.4+ MB
